# Markovian Model for steady-state throughput CUBIC and Hybla
Based on the paper by Bao Wong and Leung.
First we try to replicate their model

In [2]:
import numpy as np

N = 100 # Resolution of the discretisation
K = 100 # Total simulation time.


First we try to just compute different stationnary distributions for different parameters.

In [ ]:
class CCA_CUBIC_MarkovChain:
    def __init__(self, N=100, W = 1, trans_err = 0.01) -> None:
        self.N = N
        self.W = W
        self.trans_err = trans_err
        self.M = np.zeros(N)
        self.pi = np.ones(N)/N # Stationnary distribution

    def transition_proba_cubic(i,j):
        return np.exp(-self.trans_err)#fill in the forumla